In [ ]:
#Task-3 solution

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import time


window_width = 800
window_height = 600

catcher_point_x = window_width//2   #catcher_initial
catcher_point_y = 15
catcher_w = 115
catcher_h = 20
catcher_speed = 250
catcher_color = [1, 1, 1]

Diamond_point_x = random.randint(40, window_width-40)   #diamond_initial
Diamond_point_y = window_height-50
diamond_size = 30
diamond_speed = 90
diamond_colors= [
    [1, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [0.27, 0.34, 0.96],
    [0.02, 0.95, 0.96],
    [0.66, 0.96, 0.02],
    [0.98, 0.02, 0.86],
    [0.53, 0.1, 0.85],
    [0.85, 0.62, 0.03]
]
current_diamond_color = random.choice(diamond_colors)

Button_size = 50
Button_point_y = window_height-20

state_of_game = "play"
score = 0
l_time = time.time()

def find_zone(x1, y1, x2, y2):
    dx = x2-x1
    dy = y2-y1

    if abs(dx) >= abs(dy):
        if dx >= 0 and dy >= 0:
            return 0
        elif dx < 0 and dy >= 0:
            return 3
        elif dx < 0 and dy < 0:
            return 4
        else:
            return 7
    else:
        if dx >= 0 and dy >= 0:
            return 1
        elif dx < 0 and dy >= 0:
            return 2
        elif dx < 0 and dy < 0:
            return 5
        else:
            return 6

def convert_to_zone_0(x, y, zone):

    if zone == 0:
        return x, y
    elif zone == 1:
        return y, x
    elif zone == 2:
        return y, -x
    elif zone == 3:
        return -x, y
    elif zone == 4:
        return -x, -y
    elif zone == 5:
        return -y, -x
    elif zone == 6:
        return -y, x
    elif zone == 7:
        return x, -y

def convert_from_zone_0(x, y, zone):

    if zone == 0:
        return x, y
    elif zone == 1:
        return y, x
    elif zone == 2:
        return -y, x
    elif zone == 3:
        return -x, y
    elif zone == 4:
        return -x, -y
    elif zone == 5:
        return -y, -x
    elif zone == 6:
        return y, -x
    elif zone == 7:
        return x, -y

def midpoint_line_zone_0(x1, y1, x2, y2):

    points = []

    dx = x2 - x1
    dy = y2 - y1

    if dx == 0 :
        return[(x1, y1)]

    d = 2*dy - dx

    east = 2*dy
    north_east = 2*dy - 2*dx

    x = x1
    y = y1

    while x <= x2:
        points.append((x,y))

        if d>0:
             d+=north_east
             y+=1
        else:
            d+=east
        x+=1
    return points

def draw_line(x1, y1, x2, y2):

    if x1 == x2 and y1==y2:
        glVertex2f(x1, y1)
        return
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    zone = find_zone(x1, y1, x2, y2)

    x1_z_0, y1_z_0 = convert_to_zone_0(x1, y1, zone)
    x2_z_0, y2_z_0 = convert_to_zone_0(x2, y2, zone)

    if x1_z_0 > x2_z_0:
        x1_z_0, x2_z_0 = x2_z_0, x1_z_0
        y1_z_0, y2_z_0 = y2_z_0, y1_z_0

    points_z_0 = midpoint_line_zone_0(x1_z_0, y1_z_0, x2_z_0, y2_z_0)

    for x, y in points_z_0:
        x_new, y_new = convert_from_zone_0(x, y, zone)
        glVertex2f(x_new, y_new)


def Catcher_Draw( x, y, width, height, color):

    glColor3f(color[0], color[1], color[2])
    glPointSize(3.0)
    glBegin(GL_POINTS)

    #bottomo
    bl_x, bl_y = x-width//2+10, y
    br_x, br_y = x+width//2-10, y
    #top
    tr_x, tr_y = x+width//2, y+height
    tl_x, tl_y = x-width//2, y+height

    draw_line(tl_x, tl_y, bl_x, bl_y)
    draw_line(bl_x, bl_y, br_x, br_y)
    draw_line(br_x, br_y, tr_x, tr_y)
    draw_line(tr_x, tr_y, tl_x, tl_y)


    glEnd()

def Diamond_Draw(x, y, size, color):

    glColor3f(color[0], color[1], color[2])
    glPointSize(3.0)
    glBegin(GL_POINTS)
    t_x, t_y = x, y+size//2
    b_x, b_y = x, y-size//2
    r_x, r_y = x+size//2, y
    l_x, l_y = x-size//2, y

    draw_line(t_x,t_y, r_x, r_y)
    draw_line(r_x, r_y, b_x, b_y)
    draw_line(b_x,b_y, l_x, l_y)
    draw_line(l_x,l_y, t_x, t_y)
    glEnd()

def Restart_Button_Draw(x, y, size):

    glColor3f(0.1, 0.87, 0.87)
    glPointSize(3.0)
    glBegin(GL_POINTS)

    draw_line(x-size//3, y, x, y+size//4)
    draw_line(x-size//3, y, x, y-size//4)
    draw_line(x-size//3, y, x+size//3, y)


    glEnd()

def Play_Pause_Button(x, y):

    glColor3f(0.88, 0.88, 0.08)
    glPointSize(3.0)
    glBegin(GL_POINTS)

    if state_of_game == "paused":
      draw_line(x-10, y+10, x-10, y-10)
      draw_line(x-10, y-10, x+10, y)
      draw_line(x+10, y, x-10, y+10)


    else:
      draw_line(x-5, y-10, x-5, y+10)
      draw_line(x+5, y-10, x+5, y+10)

    glEnd()

def Exit_Button_Draw(x, y, size):

    glColor3f(1, 0, 0)
    glPointSize(3.0)
    glBegin(GL_POINTS)

    draw_line(x-size//4, y-size//4, x+size//4, y+size//4)
    draw_line(x-size//4, y+size//4, x+size//4, y-size//4)

    glEnd()

##################################################################################

def Update_Game(new_time):
    global Diamond_point_y, score, diamond_speed, state_of_game, catcher_color

    if state_of_game != "play":
        return
    Diamond_point_y -= diamond_speed*new_time

    if Collision_check(Diamond_point_x, Diamond_point_y, diamond_size, diamond_size, catcher_point_x, catcher_point_y, catcher_w, catcher_h):

        score +=1
        print(f'Score: {score}')
        diamond_speed += 12
        Spawn_New_Diamond()

    elif Diamond_point_y < -20:
        state_of_game = "game_over"
        catcher_color = [1, 0, 0]
        print(f"Game Over! Final Score: {score}")

def Reset_Game():
    global score, diamond_speed, state_of_game, catcher_color

    score = 0
    diamond_speed = 80
    state_of_game = "play"
    catcher_color = [1, 1, 1]
    Spawn_New_Diamond()


def Spawn_New_Diamond():
    global Diamond_point_x, Diamond_point_y, current_diamond_color

    Diamond_point_x = random.randint(diamond_size, window_width-diamond_size)
    Diamond_point_y =  window_height-50
    current_diamond_color = random.choice(diamond_colors)

def Collision_check(D_x, D_y, D_w, D_h, C_x, C_y, C_w, C_h):

    obj1_left = D_x-D_w//2
    obj1_bottom = D_y-D_h//2

    obj2_left = C_x-C_w//2
    obj2_bottom = C_y-C_h//2

    return(obj1_left < obj2_left+C_w and obj2_left < obj1_left+D_w and obj1_bottom < obj2_bottom+C_h and obj2_bottom < obj1_bottom+D_h)

##############################################################################
def display():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    if (state_of_game != "over" or Diamond_point_y >= 0):
        Diamond_Draw(Diamond_point_x, Diamond_point_y, diamond_size, current_diamond_color)

    Catcher_Draw(catcher_point_x, catcher_point_y, catcher_w, catcher_h, catcher_color)
    Restart_Button_Draw(30, Button_point_y, Button_size)
    Play_Pause_Button(window_width//2, Button_point_y)
    Exit_Button_Draw(window_width-30, Button_point_y, Button_size)

    glutSwapBuffers()

def idle():
    global l_time

    current_time =  time.time()
    delta_time = current_time - l_time

    if delta_time > 1.0/60.0:
        Update_Game(delta_time)
        glutPostRedisplay()
        l_time = current_time

def Mouse_Click(button, state, x, y):
    global state_of_game

    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        screen_y = window_height-y
        button_radious = Button_size//2

        if (abs(x - 30) <= button_radious and abs(screen_y-Button_point_y) <= button_radious):
             Reset_Game()

        elif(abs(x-window_width//2) <= button_radious and abs(screen_y-Button_point_y) <= button_radious):
            if state_of_game == "play":
                state_of_game = "paused"

            elif state_of_game == "paused":
                state_of_game = "play"

        elif(abs(x-(window_width-30)) <= button_radious and abs(screen_y-Button_point_y) <= button_radious):
            print(f'Final Score: {score}')
            glutLeaveMainLoop()

def Keyboard( key, x, y):
    global catcher_point_x

    if state_of_game != "play":
        return
    movement = 15

    if key == GLUT_KEY_LEFT:
        catcher_point_x -= movement
        catcher_point_x = max(catcher_w//2, catcher_point_x)
    elif key == GLUT_KEY_RIGHT:
        catcher_point_x += movement
        catcher_point_x = min(window_width - catcher_w//2, catcher_point_x)

def init():

    glClearColor(0, 0, 0, 0)
    glPointSize(3.0)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, window_width, 0, window_height)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    Spawn_New_Diamond()

    print("Catch the Diamonds")

def main():

    glutInit()
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
    glutInitWindowSize(window_width, window_height)
    glutInitWindowPosition(100, 100)
    glutCreateWindow(b"Catch The Diamonds")
    init()
    glutDisplayFunc(display)
    glutSpecialFunc(Keyboard)
    glutMouseFunc(Mouse_Click)
    glutIdleFunc(idle)
    glutMainLoop()

if __name__ == "__main__":
    main()